# LangChain 实战：航运业务聊天机器人

In [14]:
with open("real_shipping_data.txt", "r", encoding='utf-8') as f:
    real_shipping = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [15]:
from langchain.text_splitter import CharacterTextSplitter

In [16]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [17]:
docs = text_splitter.create_documents([real_shipping])

Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 144, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 129, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 170, which is longer than the specified 100
Created a chunk of size 128, which is longer than the specified 100
Created a chunk of size 149, which is longer than the specified 100
Created a chunk of size 148, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 132, which is longer than the specified 100
Created a chunk of size 138, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 135, which is longer tha

In [18]:
docs[0]

Document(page_content='[用户问题] 航运市场主要分为哪几种类型?\n[业务人员回答] 航运市场主要分为散货市场, 集装箱市场和油轮市场.')

In [19]:
len(docs)

30

### 使用 Faiss 作为向量数据库，持久化存储航运业务问答对（QA-Pair）

In [20]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

D:\soft\miniConda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [21]:
query = "有哪些船"

In [22]:
answer_list = db.similarity_search(query)

In [23]:
for ans in answer_list:
    print(ans.page_content + "\n")

[用户问题] 船舶的主要分类有哪些?
[业务人员回答] 船舶主要可以分为货船, 客船, 渔船, 特种船等.货船中还有散货船, 油轮, 集装箱船等.客船包括邮轮和渡轮.特种船包括冰区船舶, 搜救船, 拖轮等.

[用户问题] 船只遵循哪些国际法规以确保航行安全?
[业务人员回答] 只应遵循<国际海上避碰规则>(COLREGs), <国际海上人命安全公约>(SOLAS)和<海洋污染防治国际公约>(MARPOL)等.

[用户问题] 集装箱船和散货船有何不同?
[业务人员回答] 集装箱船用于运输标准化的集装箱货物, 而散货船则专门用来运输未经包装的散装货物, 如石油, 煤炭和谷物.

[用户问题] Panamax和Post-Panamax船舶有什么区别?
[业务人员回答] Panamax船舶是指最大尺寸可以穿过巴拿马运河原有的船闸的船舶.Post-Panamax船舶是指尺寸大于原巴拿马运河船闸限制的船舶, 因此无法穿越旧运河但可能适用于2016年后扩建的新运河.



In [24]:
db.save_local("real_shipping")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [25]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [26]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B1B64D5BD0>, search_kwargs={'k': 3})

In [27]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\soft\miniConda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[用户问题] 船舶的主要分类有哪些?
[业务人员回答] 船舶主要可以分为货船, 客船, 渔船, 特种船等.货船中还有散货船, 油轮, 集装箱船等.客船包括邮轮和渡轮.特种船包括冰区船舶, 搜救船, 拖轮等.

[用户问题] 船只遵循哪些国际法规以确保航行安全?
[业务人员回答] 只应遵循<国际海上避碰规则>(COLREGs), <国际海上人命安全公约>(SOLAS)和<海洋污染防治国际公约>(MARPOL)等.

[用户问题] 集装箱船和散货船有何不同?
[业务人员回答] 集装箱船用于运输标准化的集装箱货物, 而散货船则专门用来运输未经包装的散装货物, 如石油, 煤炭和谷物.



In [34]:
docs = topK_retriever.get_relevant_documents("有哪些船？")

In [35]:
for doc in docs:
    print(doc.page_content + "\n")

[用户问题] 船舶的主要分类有哪些?
[业务人员回答] 船舶主要可以分为货船, 客船, 渔船, 特种船等.货船中还有散货船, 油轮, 集装箱船等.客船包括邮轮和渡轮.特种船包括冰区船舶, 搜救船, 拖轮等.

[用户问题] 集装箱船和散货船有何不同?
[业务人员回答] 集装箱船用于运输标准化的集装箱货物, 而散货船则专门用来运输未经包装的散装货物, 如石油, 煤炭和谷物.

[用户问题] 船只遵循哪些国际法规以确保航行安全?
[业务人员回答] 只应遵循<国际海上避碰规则>(COLREGs), <国际海上人命安全公约>(SOLAS)和<海洋污染防治国际公约>(MARPOL)等.



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [36]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [37]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\soft\miniConda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`业务人员回答`

In [38]:
docs = retriever.get_relevant_documents(query)

D:\soft\miniConda\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:796: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [39]:
docs[0].page_content

IndexError: list index out of range

In [22]:
docs[0].page_content.split("[业务人员回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [23]:
ans = docs[0].page_content.split("[业务人员回答] ")[-1]

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想离医院近点"

print(sales(query))

[]


/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [28]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []



/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}